# 1/4: Data retrieval
By Niloufar Shahdoust (niloufar.shahdoust@utah.edu)

In [1]:
import os
import mat73
import numpy as np
import pandas as pd
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from visbrain.objects import BrainObj, SceneObj, SourceObj   

## required functions

In [2]:
def GetChannelLabels(pid, my_directory):


    print("patient:", pid) 
    local_dir = my_directory
    ElecMap_Path = os.path.join(local_dir, pid, "Imaging", "Registered", "Electrodes.mat")

    # Load Electrode Map
    ElecMap = mat73.loadmat(ElecMap_Path)
    ElecMap_raw = ElecMap["ElecMapRaw"]

    nan_column_index = 2 if any(
        x[2] is not None and not np.isnan(x[2]) for x in ElecMap_raw
    ) else 1

    # Filter out rows with NaN in the specified column
    ElecMap_raw = [x for x in ElecMap_raw if not np.isnan(x[nan_column_index])]
    sorted_list = sorted(ElecMap_raw, key=lambda x: int(x[nan_column_index]))
    Channel_Names = [x[0] for x in sorted_list if "m" not in x[0]]  # Exclude micro channels



    Brainnetome = mat73.loadmat(ElecMap_Path)
    Nmm_raw = pd.DataFrame(Brainnetome["ElecAtlasProjRaw"]).iloc[:, 0]
    mni_xyz = pd.DataFrame(Brainnetome["ElecXYZMNIProjRaw"]).values.tolist()

    Nmm_atlas = Nmm_raw.dropna().tolist()



    # Combine Channel Names with Networks and Atlas
    All_Information = list(zip(Channel_Names, Nmm_atlas, mni_xyz))

    # Create a DataFrame from All_Information
    df = pd.DataFrame(
        All_Information,
        columns=["Channel_Name", "Nmm_atlas", "MNI_Coordinates"]
    )

    return df


## Reading folders names

In [3]:
network_path = r'\\155.100.91.44\d\Data\preProcessed\BART_preprocessed'
all_patients = []

if os.path.exists(network_path):
    for item in os.listdir(network_path):
        item_path = os.path.join(network_path, item)
        if os.path.isdir(item_path) and item.startswith('20'):
            all_patients.append(item)
else:
    print(f"The path {network_path} does not exist or is inaccessible.")


In [4]:
output_folder = os.path.join("1_brain_visualization_data_retrieval")

for patient_id in all_patients:
    try:
        patient_df = GetChannelLabels(patient_id, network_path)
        csv_file_path = os.path.join(output_folder, f"{patient_id}.csv")
        patient_df.to_csv(csv_file_path, index=False)
        print(f"Saved {patient_id} data to {csv_file_path}")
    except Exception as e:
        print(f"Error processing patient {patient_id}: {e}")

print("All patient data has been processed and saved as CSV files.")


patient: 201810
Saved 201810 data to 1_brain_visualization_data_retrieval\201810.csv
patient: 201811
Saved 201811 data to 1_brain_visualization_data_retrieval\201811.csv
patient: 201901
Saved 201901 data to 1_brain_visualization_data_retrieval\201901.csv
patient: 201902
Saved 201902 data to 1_brain_visualization_data_retrieval\201902.csv
patient: 201902r
Saved 201902r data to 1_brain_visualization_data_retrieval\201902r.csv
patient: 201903
Saved 201903 data to 1_brain_visualization_data_retrieval\201903.csv
patient: 201905
Saved 201905 data to 1_brain_visualization_data_retrieval\201905.csv
patient: 201906
Error processing patient 201906: \\155.100.91.44\d\Data\preProcessed\BART_preprocessed\201906\Imaging\Registered\Electrodes.mat is not a MATLAB 7.3 file. Load with scipy.io.loadmat() instead.
patient: 201909
Saved 201909 data to 1_brain_visualization_data_retrieval\201909.csv
patient: 201910
Saved 201910 data to 1_brain_visualization_data_retrieval\201910.csv
patient: 201911
Saved 20